In [24]:
import pandas as pd
import requests
import csv
import time
import random
from bs4 import BeautifulSoup
from tqdm import tqdm

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36 Edg/134.0.0.0"
}
recipe_url = "https://www.10000recipe.com/recipe/list.html?order=reco&page="
url = "https://www.10000recipe.com/recipe/"
food_data = []
recipe_data = []

try:
    # --- 1. 음식 목록 긁어 오기 ---
    for i in tqdm(range(50), desc="페이지 목록 수집 중"):
        response = requests.get(recipe_url + str(i + 1), headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        food_items = soup.select('li.common_sp_list_li')

        for item in food_items:
            id_tag = item.select_one('a.common_sp_link')
            title_tag = item.select_one('div.common_sp_caption_tit')
            author_tag = item.select_one('div.common_sp_caption_rv_name')
            view_tag = item.select_one('div.common_sp_caption_rv span.common_sp_caption_buyer')
            img_tag = item.select_one('img[src*="/cache/recipe"]')

            id_val = id_tag['href'].replace('/recipe/', '') if id_tag and 'href' in id_tag.attrs else 'None'
            title = title_tag.get_text(strip=True) if title_tag else 'None'
            author = author_tag.get_text(strip=True) if author_tag else 'None'
            view = view_tag.get_text().replace('조회수', '').strip() if view_tag else 'None'
            img = img_tag['src'] if img_tag and 'src' in img_tag.attrs else 'None'

            food_data.append({
                'ID': id_val,
                'Title': title,
                'Author': author,
                'View': view,
                'Image': img
            })
        time.sleep(random.uniform(0.5, 1))

    print(f"\n총 {len(food_data)}개의 레시피 리스트 확보.")

    # --- 2. 레시피 상세 정보 긁어 오기 ---
    for food in tqdm(food_data, desc="레시피 상세 정보 수집 중"):
        try:
            response = requests.get(url + str(food['ID']), headers=headers, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # 링크
            recipe_link = url + str(food['ID'])

            # 기본 정보 추출
            serv_tag = soup.select_one('span.view2_summary_info1')
            prep_tag = soup.select_one('span.view2_summary_info2')
            diff_tag = soup.select_one('span.view2_summary_info3')

            difficulty = diff_tag.get_text(strip=True) if diff_tag else 'None'

            if difficulty not in ['아무나', '초급']:
                continue

            serving = serv_tag.get_text(strip=True) if serv_tag else 'None'
            prep_time = prep_tag.get_text(strip=True) if prep_tag else 'None'

            main_thumb_id = soup.select_one('#main_thumbs')
            main_thumb = main_thumb_id['src'] if main_thumb_id else 'None'

            summary_in_id = soup.select_one('#recipeIntro')
            summary_in = summary_in_id.get_text(strip=True) if summary_in_id else 'None'

            # 재료 추출
            ingred_items = soup.select('div.ready_ingre3 ul')
            ingredient_text = 'None'
            condiment_text = 'None'

            if not ingred_items:
                continue
            else:
                for item in ingred_items:
                    title_tag = item.select_one('b.ready_ingre3_tt')
                    group_title = title_tag.get_text(strip=True) if title_tag else 'None'
                    ingredients = item.select('li')
                    ingredient_list = []
                    for ingredient_item in ingredients:
                        name_tag = ingredient_item.select_one('div.ingre_list_name a') or ingredient_item.select_one('div.ingre_list_name')
                        ea_tag = ingredient_item.select_one('span.ingre_list_ea')
                        i_name = name_tag.get_text(strip=True) if name_tag else ''
                        i_ea = ea_tag.get_text(strip=True) if ea_tag else ''
                        if i_name:
                            ingredient_list.append(f'{i_name}-{i_ea}')
                    joined_ingredients = ','.join(ingredient_list)
                    if '재료' in group_title or '레시피' in group_title:
                        ingredient_text = joined_ingredients
                    elif '양념' in group_title:
                        condiment_text = joined_ingredients

            # --- [수정됨] 조리 순서(Steps) 이미지 추출 로직 개선 ---
            step_divs = soup.select('div.view_step_cont')
            step_list = []

            skip_recipe = False
            for i, step_div in enumerate(step_divs):
                desc_tag = step_div.select_one('div.media-body')
                desc = desc_tag.get_text(strip=True) if desc_tag else "None"

                # 수정 포인트: src 속성에 '/recipe/'가 포함된 이미지를 찾음 (구조 무관)
                img_tag = step_div.select_one('img[src*="/recipe/"]')

                # 만약 위 방식으로 안 잡히면 일반 img 태그 시도 (아이콘 등 제외 필요 시 필터링)
                if not img_tag:
                    img_tag = step_div.select_one('img')

                # 조리순서에 이미지 없으면 스킵
                img_src = img_tag['src'] if img_tag else "None"
                if img_src == 'None':
                    skip_recipe = True
                    break

                step_str = f"{i+1}단계||{desc}||{img_src}"
                step_list.append(step_str)

            if skip_recipe:
                continue
            all_steps_text = "\n".join(step_list)

            recipe_data.append({
                'Recipe_Link': recipe_link,
                'Title': food['Title'],
                'Serving': serving,
                'Preparation_Time': prep_time,
                'Difficulty': difficulty,
                'Main_Thumbnail': main_thumb,
                'Summary_In': summary_in,
                'Ingredient': ingredient_text,
                'Condiment': condiment_text,
                'Steps': all_steps_text
            })

            time.sleep(random.uniform(0.5, 1))

        except Exception as e:
            print(f"ID {food['ID']} 처리 중 에러 발생: {e}")
            continue

    print("\n--- 추출 결과 확인 (상위 1개) ---")
    if recipe_data:
        print(recipe_data[0])

except Exception as e:
    print(f"전체 프로세스 중 치명적 오류: {e}")

# --- 3. CSV 저장 ---
fieldnames = ['Recipe_Link', 'Title', 'Serving', 'Preparation_Time', 'Difficulty', 'Main_Thumbnail', 'Summary_In', 'Ingredient', 'Condiment', 'Steps']

filename = 'recipe_data.csv'
with open(filename, 'w', newline='', encoding='utf-8-sig') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(recipe_data)

print(f"'{filename}' 파일 저장 완료.")

페이지 목록 수집 중: 100%|██████████| 50/50 [00:58<00:00,  1.18s/it]



총 2000개의 레시피 리스트 확보.


레시피 상세 정보 수집 중:  15%|█▍        | 297/2000 [06:45<29:28,  1.04s/it]

ID 7027794 처리 중 에러 발생: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


레시피 상세 정보 수집 중: 100%|██████████| 2000/2000 [43:20<00:00,  1.30s/it]


--- 추출 결과 확인 (상위 1개) ---
{'Recipe_Link': 'https://www.10000recipe.com/recipe/7069205', 'Title': '양파 요리, 굴소스 양파계란볶음 레시피', 'Serving': '2인분', 'Preparation_Time': '20분 이내', 'Difficulty': '아무나', 'Main_Thumbnail': 'https://recipe1.ezmember.co.kr/cache/recipe/2026/01/03/1c9d9e86727d9e3fa5ae33c744fcceac1.jpg', 'Summary_In': '양파 요리, 굴소스 양파계란볶음 레시피 입니다.', 'Ingredient': '계란-3알,양파-1개,대파-1대,식용유-2T,굴소스-1T,깨소금-1T,참기름-1T', 'Condiment': 'None', 'Steps': '1단계||대파는 원하는 크기로 썰어주시고, 양파는 얇게 슬라이스 해주세요.||https://recipe1.ezmember.co.kr/cache/recipe/2026/01/03/4bc4bf520fb4be6178c0f1c86ea17a151.jpg\n2단계||계란 3알을 깨서 잘 풀어 준비합니다.||https://recipe1.ezmember.co.kr/cache/recipe/2026/01/03/9b89fc22ca45d5d7210ab3655e397ade1.jpg\n3단계||달군 프라이팬에 식용유 2T 가량 둘러주세요.||https://recipe1.ezmember.co.kr/cache/recipe/2026/01/03/1d6ba1819ed98e5bde672237b5693cb11.jpg\n4단계||썰어놓은 대파와 양파를 모두 넣어 볶아 줍니다.||https://recipe1.ezmember.co.kr/cache/recipe/2026/01/03/e95af9f903e7e82075d47ed6dc9347101.jpg\n5단계||중불에서 약 2~3분 정도 볶아 준다음 프라이팬의 한쪽으로 양파와 대파를

In [25]:
df = pd.read_csv('recipe_data.csv')
df

,Recipe_Link,Title,Serving,Preparation_Time,Difficulty,Main_Thumbnail,Summary_In,Ingredient,Condiment,Steps
0,https://www.10000recipe.com/recipe/7069205,"양파 요리, 굴소스 양파계란볶음 레시피",2인분,20분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,"양파 요리, 굴소스 양파계란볶음 레시피 입니다.","계란-3알,양파-1개,대파-1대,식용유-2T,굴소스-1T,깨소금-1T,참기름-1T",NaN,"1단계||대파는 원하는 크기로 썰어주시고, 양파는 얇게 슬라이스 해주세요.||htt..."
1,https://www.10000recipe.com/recipe/7065887,굴소스 팽이버섯 계란 덮밥 만들기 한 그릇 레시피,1인분,10분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,바쁜 날에도 10분 만에 완성되는 굴소스 팽이버섯 계란 덮밥 만들기 한 그릇 레시피...,"밥-1공기,팽이버섯-1봉지,계란-2개,대파-","식용유-1T,간장-1T,굴소스-1T,물-3T,참기름-1T",1단계||재료 준비||https://recipe1.ezmember.co.kr/cac...
2,https://www.10000recipe.com/recipe/7009944,오이고추된장무침 (사계절 먹기좋은 밑반찬 고추무침 5분 레시피),4인분,5분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,오이고추로 만든 밥반찬 밑반찬입니다 불 없이 초간단하게 만들 수 있는 레시피입니다,오이고추-7개,"된장-1스푼,고추장-0.5스푼,참기름-1스푼,올리고당-0.5스푼,통깨-1스푼","1단계||볼에 된장 양념을 모두 넣고 골고루 섞어준다믹싱볼,숟가락||https://..."
3,https://www.10000recipe.com/recipe/6993517,새송이버섯버터굴소스볶음 간단반찬,2인분,5분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,미니새송이버섯으로 만든 간단반찬입니다,"미니새송이버섯-1팩,버터-20g,굴소스-1T,진간장-0.5T,통깨-약간",NaN,1단계||새송이버섯은 길게 편 썰어 주세요||https://recipe1.ezmem...
4,https://www.10000recipe.com/recipe/7025833,매운빨간콩나물무침 레시피 매일반찬,4인분,15분 이내,초급,https://recipe1.ezmember.co.kr/cache/recipe/20...,큰아이가 오랜만에 요청한 매운빨간콩나물무침입니다.,"콩나물-300g,대파-1/2대,고추가루-1.5T,맛소금-1/3T,멸치액젓-1T,국간...",NaN,"1단계||콩나물 300G을 잘 세척해 끓는 물에 넣고 3~4분정도 삶아줍니다.볼,채..."
...,...,...,...,...,...,...,...,...,...,...
1805,https://www.10000recipe.com/recipe/6929581,"한그릇요리, 시금치 갈릭 새우 볶음밥",1인분,10분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,한그릇 요리로 즐겨도 되고반찬으로 먹어도 좋아요 ^-^!!새우에는 단백질이 풍부하고...,"시금치-1줌,새우-1/2컵,밥-1그릇,식용유-3큰술","진간장-1큰술,다진마늘-1큰술,통깨-1큰술,소금-약간","1단계||새우 껍질이 있는경우엔껍질을 먼저 제거해줍니다.생새우, 칵테일새우, 새우살..."
1806,https://www.10000recipe.com/recipe/6947570,새우 오이 볶음 다이어트 도시락 반찬,2인분,30분 이내,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,담백함이 가득해자극적이지 않지만 자꾸 끌리는 맛꼬들꼬들 쫄깃쫄깃 식감까지 좋아,"오이-1.5개,칵테일 새우-2줌,대파-약간,청양고추-1개","소금-,다진 마늘-1T,굴 소스-1t,후춧가루-,맛술-,깨-",1단계||새우에 밑간을 해 줍니다.칵테일 새우는 해동 후 물에 헹궈 준 후 꼬리를 ...
1807,https://www.10000recipe.com/recipe/7039703,콩나물국,4인분,15분 이내,초급,https://recipe1.ezmember.co.kr/cache/recipe/20...,찬바람이 불고 겨울이 오면서 우리집에는 국이 없으면 밥을 먹지 못하는 남편 덕분에 ...,"콩나물-1봉지,다진마늘-1스푼,대파-적당량,국간장-1스푼,소금-약간,코인육수-2알",NaN,1단계||콩나물국을 소개합니다.||https://recipe1.ezmember.co...
1808,https://www.10000recipe.com/recipe/6879004,한우로 끓이는 얼큰한 경상도식 소고기국,2인분,30분 이내,초급,https://recipe1.ezmember.co.kr/cache/recipe/20...,한우로 끓인 얼큰하고 시원한 경상도식 소고기국해장용으로도 좋아요,"한우 양지-300g,무-1/4개,콩나물-1줌,대파-1대,국간장-4T,진간장-1T,참...",NaN,1단계||소고기는 한우 양지로 준비해주세요.찬물에 30분정도 담가 핏물을 충분히 빼...


In [22]:
df['Steps']

0     [Step1]-대파는 원하는 크기로 썰어주시고, 양파는 얇게 슬라이스 해주세요.||...
1     [Step1]-재료 준비||IMG:https://recipe1.ezmember.co...
2     [Step1]-볼에 된장 양념을 모두 넣고 골고루 섞어준다믹싱볼,숟가락||IMG:h...
3     [Step1]-새송이버섯은 길게 편 썰어 주세요||IMG:https://recipe...
4     [Step1]-콩나물 300G을 잘 세척해 끓는 물에 넣고 3~4분정도 삶아줍니다....
                            ...                        
71    [Step1]-참치캔은 체에 받쳐 기름을 빼준다||IMG:https://recipe...
72    [Step1]-아보카도를 반으로 갈라 이쁘게 잘라줍니다.도마,조리용나이프||IMG:...
73    [Step1]-납작어묵과 프랑크소세지를 준비해주세요||IMG:https://reci...
74    [Step1]-팽이버섯 밑동 자르고 물로 가볍게 흔들어 씻은 후 큼직큼직 하게 찢기...
75    [Step1]-프라임 꼬치구이 프랑크를 먹기좋은 크기로 썰어주세요.||IMG:htt...
Name: Steps, Length: 76, dtype: object